# Automated ordering, download, indexing of Landsat USGS data for AGDCv2

### Import the required libraries

In [1]:
import os, json, requests, time
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import gzip

## Register with ESPA and enter your credentials below https://espa.cr.usgs.gov

In [67]:
username = 'simonaoliver'
password = 'xxxxxxxxxx'
email = 'simon.oliver@ga.gov.au'
espa_bulk_downloader = os.path.abspath('C:\datacube\code\agdc-v2\code\espa-bulk-downloader\download_espa_order.py')
datacube_home = os.path.abspath('C:\datacube\code\agdc-v2')
data_dir = os.path.abspath('D:\input\Hyderabad_India')
working_dir = os.path.abspath("C:\datacube\work")
utils = 'utils\\usgslsprepare.py'

## Determine which WRS path/rows intersect the area of interest

### Define the AOI

In [68]:
ul_lon, ul_lat = 77.83, 17.84 # upper left longitude, latitude
lr_lon, lr_lat = 78.00, 17.67 # lower right longitude, latitude
date_start, date_end = "2015-01-11"  , "2016-12-12" # start date and end date for time range selection  
sensor_list=["olitirs8", "tm5", "etm7"] # list of sensors to use

polygon_list = [[ul_lon, ul_lat], [lr_lon, ul_lat],[lr_lon, lr_lat],[ul_lon, lr_lat],[ul_lon, ul_lat]]

wrs_query = 'http://api.remotepixel.ca/wrs2tiles?search=poly:'+str(polygon_list)

### Determine which WRS2 path/rows cover the AOI

In [22]:
post_query = requests.get(wrs_query)
wrs_search_result = json.loads(post_query.text)

path_row = []
for item in wrs_search_result['results']:
    path_row.append(str(item['path'])+"_"+str(item['row']))
path_row

['144_48']

### Get the latest Level 1 inventory from USGS to find the available scenes 

In [5]:
def download_file(url, output_dir):
    local_filename = os.path.join(working_dir,url.split('/')[-1])
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename

In [6]:
landsat_csv_list = ["https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_8.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_ETM.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_ETM_SLC_OFF.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_TM-1980-1989.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_TM-1990-1999.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_TM-2000-2009.csv.gz",\
                    "https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_TM-2010-2012.csv.gz"
                   ]

#### Download the inventory data from USGS

In [10]:
for csv in landsat_csv_list:
    download_file(csv, working_dir)

In [11]:
landsat_csv = []
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_8.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_ETM.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_ETM_SLC_OFF.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_TM-1980-1989.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_TM-1990-1999.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_TM-2000-2009.csv.gz'))
landsat_csv.append(os.path.join(working_dir, 'LANDSAT_TM-2010-2012.csv.gz'))

#### Unzip inventory files

In [12]:
## TODO read directly to json obj
for csv in landsat_csv:
    inF = gzip.open(csv, 'rb')
    print("Unzipping "+csv)
    outF = open((csv.replace('.gz','')), 'wb')
    outF.write( inF.read() )
    inF.close()
    outF.close()

Unzipping C:\datacube\work\LANDSAT_8.csv.gz
Unzipping C:\datacube\work\LANDSAT_ETM.csv.gz
Unzipping C:\datacube\work\LANDSAT_ETM_SLC_OFF.csv.gz
Unzipping C:\datacube\work\LANDSAT_TM-1980-1989.csv.gz
Unzipping C:\datacube\work\LANDSAT_TM-1990-1999.csv.gz
Unzipping C:\datacube\work\LANDSAT_TM-2000-2009.csv.gz
Unzipping C:\datacube\work\LANDSAT_TM-2010-2012.csv.gz


In [55]:
scene_list = []
time_list = []
landsat_inventory=[os.path.join(working_dir,'LANDSAT_8.csv'),\
                   os.path.join(working_dir,'LANDSAT_ETM.csv'),\
                   os.path.join(working_dir,'LANDSAT_ETM_SLC_OFF.csv'),\
                   os.path.join(working_dir,'LANDSAT_TM-1980-1989.csv'),\
                   os.path.join(working_dir,'LANDSAT_TM-1990-1999.csv'),\
                   os.path.join(working_dir,'LANDSAT_TM-2000-2009.csv'),\
                   os.path.join(working_dir,'LANDSAT_TM-2010-2012.csv')
                  ]

In [56]:
for collection in landsat_inventory:
    data_inventory = pd.read_csv(collection , usecols=['acquisitionDate', "sceneID", "path", "row"]) # limit the columns to only the ones we need
    data_inventory["path_row"] = data_inventory["path"].map(str) + "_" + data_inventory["row"].map(str)
    data_inventory['acquisitionDate'] = data_inventory['acquisitionDate'].apply(pd.to_datetime)
    data_inventory = data_inventory.loc[(data_inventory['acquisitionDate'] > pd.tslib.Timestamp(date_start+' 00:00:00'))& (data_inventory['acquisitionDate'] <= pd.tslib.Timestamp(date_end+' 00:00:00'))]
    scene_list.append(data_inventory.loc[data_inventory['path_row'].isin(path_row)]['sceneID'].tolist())

In [57]:
scene_list

[['LC81440482016336LGN00',
  'LC81440482016320LGN00',
  'LC81440482016304LGN00',
  'LC81440482016288LGN00',
  'LC81440482016272LGN00',
  'LC81440482016256LGN00',
  'LC81440482016240LGN00',
  'LC81440482016224LGN00',
  'LC81440482016208LGN00',
  'LC81440482016192LGN00',
  'LC81440482016176LGN00',
  'LC81440482016160LGN00',
  'LC81440482016144LGN00',
  'LC81440482016128LGN00',
  'LC81440482016112LGN00',
  'LC81440482016096LGN00',
  'LC81440482016080LGN00',
  'LC81440482016064LGN00',
  'LC81440482016048LGN00',
  'LC81440482016032LGN00',
  'LC81440482016016LGN00',
  'LC81440482015365LGN00',
  'LC81440482015349LGN00',
  'LC81440482015333LGN00',
  'LC81440482015317LGN00',
  'LC81440482015301LGN00',
  'LC81440482015285LGN00',
  'LC81440482015269LGN00',
  'LC81440482015253LGN00',
  'LC81440482015237LGN00',
  'LC81440482015221LGN00',
  'LC81440482015205LGN00',
  'LC81440482015189LGN00',
  'LC81440482015173LGN00',
  'LC81440482015157LGN00',
  'LC81440482015141LGN00',
  'LC81440482015125LGN00',
 

## Submit an ESPA order for the SR product for the scene list

In [59]:
def is_json(myjson):
  try:
    json_object = json.loads(myjson)
  except ValueError, e:
    return False
  return True

In [97]:
json_order=[]
for sensor in sensor_list:
    order_details={sensor: {"products": ["sr"]}, "format": "gtiff" ,"resize": {"pixel_size": 30, "pixel_size_units": "meters"}}
    
    if (len(scene_list[sensor_list.index(sensor)]) == 0):
        print("no inputs for this sensor: ", sensor)
    else:
        order_details[sensor]["inputs"] = scene_list[sensor_list.index(sensor)]
        json_order.append(json.dumps(order_details))
json_order

('no inputs for this sensor: ', 'tm5')


['{"olitirs8": {"inputs": ["LC81440482016336LGN00", "LC81440482016320LGN00", "LC81440482016304LGN00", "LC81440482016288LGN00", "LC81440482016272LGN00", "LC81440482016256LGN00", "LC81440482016240LGN00", "LC81440482016224LGN00", "LC81440482016208LGN00", "LC81440482016192LGN00", "LC81440482016176LGN00", "LC81440482016160LGN00", "LC81440482016144LGN00", "LC81440482016128LGN00", "LC81440482016112LGN00", "LC81440482016096LGN00", "LC81440482016080LGN00", "LC81440482016064LGN00", "LC81440482016048LGN00", "LC81440482016032LGN00", "LC81440482016016LGN00", "LC81440482015365LGN00", "LC81440482015349LGN00", "LC81440482015333LGN00", "LC81440482015317LGN00", "LC81440482015301LGN00", "LC81440482015285LGN00", "LC81440482015269LGN00", "LC81440482015253LGN00", "LC81440482015237LGN00", "LC81440482015221LGN00", "LC81440482015205LGN00", "LC81440482015189LGN00", "LC81440482015173LGN00", "LC81440482015157LGN00", "LC81440482015141LGN00", "LC81440482015125LGN00", "LC81440482015109LGN00", "LC81440482015093LGN00"

In [61]:
# TODO - determine if there are limits on the number of scenes per order and factor that into jobs are submitted

## Submit your order

In [103]:
for json_item in json_order:
    print('https://espa.cr.usgs.gov/api/v0/order', 'data=',json_item, 'auth=(',username, password,')')
    place_order = requests.post('https://espa.cr.usgs.gov/api/v0/order', data=json_item, auth=(username, password))
    print(place_order.text)

('https://espa.cr.usgs.gov/api/v0/order', 'data=', '{"olitirs8": {"inputs": ["LC81440482016336LGN00", "LC81440482016320LGN00", "LC81440482016304LGN00", "LC81440482016288LGN00", "LC81440482016272LGN00", "LC81440482016256LGN00", "LC81440482016240LGN00", "LC81440482016224LGN00", "LC81440482016208LGN00", "LC81440482016192LGN00", "LC81440482016176LGN00", "LC81440482016160LGN00", "LC81440482016144LGN00", "LC81440482016128LGN00", "LC81440482016112LGN00", "LC81440482016096LGN00", "LC81440482016080LGN00", "LC81440482016064LGN00", "LC81440482016048LGN00", "LC81440482016032LGN00", "LC81440482016016LGN00", "LC81440482015365LGN00", "LC81440482015349LGN00", "LC81440482015333LGN00", "LC81440482015317LGN00", "LC81440482015301LGN00", "LC81440482015285LGN00", "LC81440482015269LGN00", "LC81440482015253LGN00", "LC81440482015237LGN00", "LC81440482015221LGN00", "LC81440482015205LGN00", "LC81440482015189LGN00", "LC81440482015173LGN00", "LC81440482015157LGN00", "LC81440482015141LGN00", "LC81440482015125LGN00"

In [100]:
place_order.text

u'{\n  "message": {\n    "1 validation errors:": [\n      "Value {\'sr\': [u\'le71440482016152sgi00\']} for field \'etm7.products\' Requested products are restricted by date"\n    ]\n  },\n  "status": 400\n}'

In [ ]:
#!curl --user simonaoliver:xxxxx -d '{"olitirs8": {"inputs": ["LC81440482016336LGN00", "LC81440482016320LGN00", "LC81440482016304LGN00", "LC81440482016288LGN00", "LC81440482016272LGN00", "LC81440482016256LGN00", "LC81440482016240LGN00", "LC81440482016224LGN00", "LC81440482016208LGN00", "LC81440482016176LGN00"],"products": ["sr"]}, "format": "gtiff", "resize": {"pixel_size": 30, "pixel_size_units": "meters"} }' https://espa.cr.usgs.gov/api/v0/order

In [101]:
current_orders = requests.get('https://espa.cr.usgs.gov/api/v0/list-orders/'+email, auth=(username, password))
current_orders.json()['orders']
#curl --user username:password https://espa.cr.usgs.gov/api/v0/list-orders/production@email.com

[u'simon.oliver@ga.gov.au-12032016-233815-705',
 u'simon.oliver@ga.gov.au-12042016-032108-341',
 u'simon.oliver@ga.gov.au-0101602113932',
 u'simon.oliver@ga.gov.au-0101511213509',
 u'simon.oliver@ga.gov.au-0101508269699',
 u'simon.oliver@ga.gov.au-0101411273872',
 u'simon.oliver@ga.gov.au-0101411199182']

In [102]:
for order in current_orders.json()['orders']:
    order_status = requests.get('https://espa.cr.usgs.gov/api/v0/order/'+order, auth=(username, password))
    status = order_status.json()['status']
    print(status)

purged
complete
purged
purged
purged
purged
purged


In [ ]:
for order_status in current_orders.json()['orders']:
    while order_status != 'complete' or order_status != 'purged':
        print("waiting for order to complete...checking again in 24hrs")
        time.sleep(86400)
        Print("checking status again")
        current_order_status = requests.get('https://espa.cr.usgs.gov/api/v0/order/simon.oliver@ga.gov.au-12042016-032108-341', auth=(username, password))
        print(current_order_status.json()['status'])
    print(current_order_status.json()['status'])

#### once the order status shows complete - download the data

In [ ]:
if current_order_status.json()['status'] == 'complete'
    !python $espa_bulk_downloader -e $email -o ALL -d $data_directory -u $username -p $password    

In [ ]:
# organise into scene folders then create a folder for each zip archive and unzip the archive within it
#$ for i in `ls /d/input/Hyderabad_India/*/LT5*gz`; do j=`echo $i | cut -c -58`;  tar -xvzf $i -C $j; done


### Prepare the data 

In [ ]:
ledaps_prepare_script=os.path.join(datacube_home, utils)
print(ledaps_prepare_script)

In [ ]:
data_list = [x[0] for x in os.walk(data_dir)]

In [ ]:
data_list

In [ ]:
for item in data_list[1:]:
    !python "C:\\datacube\\code\\agdc-v2\\utils\\usgslsprepare.py" $item

### Add the product configurations to the datacube database

In [ ]:
import os
ls5_config=os.path.abspath("C:\\datacube\\code\\agdc-v2\\docs\\config_samples\\dataset_types\\ls5_scenes.yaml")
ls7_config=os.path.abspath("C:\\datacube\\code\\agdc-v2\\docs\\config_samples\\dataset_types\\ls7_scenes.yaml")
ls8_config=os.path.abspath("C:\\datacube\\code\\agdc-v2\\docs\\config_samples\\dataset_types\\ls8_scenes.yaml")

In [ ]:
# Remove old datacube db instances if they exist and create a new
## Weird Ubuntu config issue fixed with sudo ln -s /var/run/postgresql/.s.PGSQL.5432 /tmp/.s.PGSQL.5432
!dropdb datacube
!createdb datacube 
# the database to prepare to product addition
!datacube system init


In [ ]:
# add the product configurations to enable indexing of prepared data
!datacube -v product add $ls5_config
!datacube -v product add $ls7_config
!datacube -v product add $ls8_config
!datacube system check

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import datacube
from datacube.model import Range
from datetime import datetime
dc = datacube.Datacube(app='dc-example')
from datacube.storage import masking
from datacube.storage.masking import mask_valid_data as mask_invalid_data
import pandas
import xarray
import numpy

In [ ]:
import folium
from IPython.display import display
import geopandas
from shapely.geometry import mapping
from shapely.geometry import MultiPolygon
import rasterio
import shapely.geometry
import shapely.ops
from functools import partial
import pyproj
from datacube.model import CRS

### Add datasets for the configured products

In [ ]:
for item in data_list[1:]:
    item=os.path.join(item,'agdc-metadata.yaml')
    if os.path.isfile(item):
        !datacube -v dataset add $item --auto-match

In [ ]:
years = list(range(1987,2017))
#products = dc.list_products().name
products=['ls5_ledaps_scene', 'ls7_ledaps_scene', 'ls8_ledaps_scene']
for year in years:
    for item in products:
        datasets = dc.index.datasets.count(product=item, time=Range(datetime(year, 1, 1), datetime(year+1, 1, 1)))
        print(year, item, datasets)
    print("*****************************")

In [ ]:
datacube.__version__

In [ ]:
def datasets_union(dss, inputcrs):
    thing = shapely.ops.unary_union([shapely.geometry.Polygon(ds.extent.points) for ds in dss])
    return shapely.geometry.shape(rasterio.warp.transform_geom(inputcrs, 'EPSG:4326', shapely.geometry.mapping(thing)))

In [ ]:
import random
def plot_folium(shapes):

    mapa = folium.Map(location=[17.3850,78.4867], zoom_start=8)
    colors=['#00ff00', '#ff0000', '#00ffff', '#ffffff', '#000000', '#ff00ff']
    for shape in shapes:
        style_function = lambda x: {'fillColor': '#000000' if x['type'] == 'Polygon' else '#00ff00', 
                                   'color' : random.choice(colors)}
        poly = folium.features.GeoJson(mapping(shape), style_function=style_function)
        mapa.add_children(poly)
    display(mapa)

In [ ]:
plot_folium([datasets_union(dc.index.datasets.search_eager(product='ls5_ledaps_scene'), 'EPSG:32644'),\
             datasets_union(dc.index.datasets.search_eager(product='ls7_ledaps_scene'), 'EPSG:32644'),\
             datasets_union(dc.index.datasets.search_eager(product='ls8_ledaps_scene'), 'EPSG:32644')])

In [ ]:
dc.list_measurements()

In [ ]:
landsat_rgb_params={'ls8olitirs': {'red': 'sr_band4', 'green': 'sr_band3', 'blue': 'sr_band2', 'product':'ls8_ledaps_scene'},\
                   'ls7etm': {'red': 'sr_band3', 'green': 'sr_band2', 'blue': 'sr_band1', 'product':'ls7_ledaps_scene'},\
                   'ls5tm': {'red': 'sr_band3', 'green': 'sr_band2', 'blue': 'sr_band1', 'product': 'ls5_ledaps_scene'}}

In [ ]:
for key in landsat_rgb_params.keys():
    print(landsat_rgb_params[key])

## CHANGE THE SENSOR TO EXECUTE

In [ ]:
sensor = 'ls8olitirs' #ls8olitirs, ls7etm, ls5tm

In [ ]:
!set GDAL_DATA

In [ ]:
sr = dc.load(product=landsat_rgb_params[sensor]['product'], 
              #x=(78.40, 78.57), y=(17.36, 17.52),
              x=(77.83, 78.00), y=(17.67, 17.84),
              measurements=[landsat_rgb_params[sensor]['red'], landsat_rgb_params[sensor]['green'], landsat_rgb_params[sensor]['blue']],
              output_crs='EPSG:32644',resolution=(-30,30))
sr = sr.where(sr != -9999)

In [ ]:
pq = dc.load(product=landsat_rgb_params[sensor]['product'], 
              #x=(78.40, 78.57), y=(17.36, 17.52),
              x=(77.83, 78.00), y=(17.67, 17.84),
              measurements=['cfmask'],
              output_crs='EPSG:32644',resolution=(-30,30)).cfmask

In [ ]:
pandas.DataFrame.from_dict(masking.get_flags_def(pq), orient='index')

In [ ]:
water = masking.make_mask(pq, cfmask ='water')
water.sum('time').plot(cmap='nipy_spectral')

In [ ]:
sr

In [ ]:
mask = masking.make_mask(pq, cfmask ='cloud')
mask = abs(mask*-1+1)
sr_cloudfree = sr.where(mask)
mask = masking.make_mask(pq, cfmask ='shadow')
mask = abs(mask*-1+1)
sr_cloudfree = sr_cloudfree.where(mask)

## Preliminary view of outputs

In [ ]:
sr_cloudfree = sr

In [ ]:
sr_cloudfree[landsat_rgb_params[sensor]['red']].plot(col='time', col_wrap=6, robust='True')

In [ ]:
## From http://scikit-image.org/docs/dev/auto_examples/plot_equalize.html
import numpy as np
from skimage import data, img_as_float
from skimage import exposure
from matplotlib import pyplot as plt

In [ ]:
# determine the clip parameters for a target clear (cloud free image) - identified through the index provided
def get_p2_p98(rgb, red, green, blue, index):

    r = np.nan_to_num(np.array(rgb.data_vars[red][index]))
    g = np.nan_to_num(np.array(rgb.data_vars[green][index]))
    b = np.nan_to_num(np.array(rgb.data_vars[blue][index]))
  
    rp2, rp98 = np.percentile(r, (2, 98))
 
    gp2, gp98 = np.percentile(g, (2, 98))
    
    bp2, bp98 = np.percentile(b, (2, 98))

    return(rp2, rp98, gp2, gp98, bp2, bp98)

In [ ]:
def plot_rgb(rgb, rp2, rp98, gp2, gp98, bp2, bp98, red, green, blue, index):

    r = np.nan_to_num(np.array(rgb.data_vars[red][index]))
    g = np.nan_to_num(np.array(rgb.data_vars[green][index]))
    b = np.nan_to_num(np.array(rgb.data_vars[blue][index]))

    r_rescale = exposure.rescale_intensity(r, in_range=(rp2, rp98))
    g_rescale = exposure.rescale_intensity(g, in_range=(gp2, gp98))
    b_rescale = exposure.rescale_intensity(b, in_range=(bp2, bp98))

    rgb_stack = np.dstack((r_rescale,g_rescale,b_rescale))
    img = img_as_float(rgb_stack)

    return(img)

In [ ]:
##### include an index here for the timeslice with representative data for best stretch of time series
# don't run this to keep the same limits as the previous sensor
rp2, rp98, gp2, gp98, bp2, bp98 = get_p2_p98(sr_cloudfree,landsat_rgb_params[sensor]['red'],landsat_rgb_params[sensor]['green'],landsat_rgb_params[sensor]['blue'], 1)
rp2, rp98, gp2, gp98, bp2, bp98 = (300.0, 2000.0, 300.0, 2000.0, 300.0, 2000.0)
print(rp2, rp98, gp2, gp98, bp2, bp98)

In [ ]:
import re
from IPython.display import clear_output
non_decimal = re.compile(r'[^\d.]+')
index = 0

for timeslice in sr_cloudfree.time.to_dict()['data']:
    print timeslice
    time_filename = non_decimal.sub('', str(timeslice))
    plt.figure(num=None, dpi=600, facecolor='w')
    #plt.figsize=(10.0, 10.0)
    #plt.annotate(str(timeslice)+' '+sensor, xy=(100, 100), xytext=(10, 20), color='red')
    #plt.tight_layout()
    plt.imshow(plot_rgb(sr_cloudfree,rp2, rp98, gp2, gp98, bp2, bp98,landsat_rgb_params[sensor]['red'],\
                        landsat_rgb_params[sensor]['green'], landsat_rgb_params[sensor]['blue'], index),interpolation='nearest')

    plt.savefig(time_filename+'.png', transparent=True, bbox_inches='tight', \
                        pad_inches=0, dpi=600)
    index = index+1
    plt.close()
    clear_output()

In [ ]:
import matplotlib.animation as animation
import numpy as np
from pylab import *

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML
HTML(anim.to_html5_video())